In [4]:
import pickle# для манипуляц со словариком пикл
import numpy as np
import pandas as pd
#from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack # добавит новый столбец не разряженный к разряженной матрице
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [18]:
pip install IPyton

Note: you may need to restart the kernel to use updated packages.


  ERROR: Could not find a version that satisfies the requirement IPyton (from versions: none)
ERROR: No matching distribution found for IPyton
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('./train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('./test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
#print(train_df[times])

train_df[times] = train_df[times].apply(pd.to_datetime).fillna(0)# приведет к формату даты Data Frame можно указать при загрузке файла!!
test_df[times] = test_df[times].apply(pd.to_datetime).fillna(0)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки



In [460]:
train_df['time1'].apply(lambda ts: ts.hour)
trrr = train_df['time2'].apply(lambda t: t.date)
trr2 = train_df['time1']
#print(train_df.time1,' --- ' , train_df.time2)
print(trrr, trr2)

AttributeError: 'int' object has no attribute 'date'

In [6]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]# список генерир с 1 по 10 имен site1 ид

train_df[sites] = train_df[sites].fillna(0).astype('int')#.to_csv('train_text.txt', sep=' ',index = None,header=None)# забиваем пустые значения нолями
test_df[sites] =   test_df[sites].fillna(0).astype('int')#.to_csv('test_text.txt', sep=' ',index = None, header=None)

# загрузим словарик сайтов
with open(r"./site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
#'www.abmecatronique.com': 25075, 'groups.live.com': 13997,

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), # ['www.abmecatronique.com', 'groups.live.com', ...]
                          index=list(site_dict.values()), #[25075, 13997, 42436, 30911,]
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()# ДРУГОЙ DATA FRAME


всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [7]:
#['www.abmecatronique.com', 'groups.live.com', 
# наша целевая переменная
y_train = train_df['target'] # otveti

# объединенная таблица обуч и тестовой
full_df = pd.concat([train_df.drop('target', axis=1), test_df])# удаляем ответы и объединяем с тестовой выборкой 

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]
idx_split
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,0,0,0,0,0,0,0,0,0,0,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,945,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,946,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,952,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22
242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,2013-01-12 08:50:24,946,2013-01-12 08:50:24,953,2013-01-12 08:50:24,955,2013-01-12 08:50:24,946,2013-01-12 08:50:25,947,2013-01-12 08:50:25
57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,2013-01-12 08:50:26,947,2013-01-12 08:50:27,953,2013-01-12 08:50:27,946,2013-01-12 08:50:27,953,2013-01-12 08:50:28,1033,2013-01-12 08:50:28
240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,2013-01-12 08:50:29,954,2013-01-12 08:50:29,946,2013-01-12 08:50:30,956,2013-01-12 08:50:30,957,2013-01-12 08:50:31,956,2013-01-12 08:50:31
210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,2013-01-12 08:50:33,954,2013-01-12 08:50:33,946,2013-01-12 08:50:34,946,2013-01-12 08:50:35,946,2013-01-12 08:50:36,948,2013-01-12 08:50:36


In [66]:
sites = ['site%s' % i for i in range(1,11)] #генерим список сайеов от  1 до 10

full_sites = full_df[sites] # новый DATA  FRAME
full_sites10 = full_sites
full_sites10

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947
242171,952,947,953,946,947,946,953,955,946,947
57157,953,947,946,953,955,947,953,946,953,1033
240201,946,947,954,953,946,954,946,956,957,956
210686,946,956,946,946,955,954,946,946,946,948


In [6]:
from scipy.sparse import csr_matrix
## последовательность с индексами
sites_flatten = full_sites10.values.flatten() #  [  56.   55.   nan ... 1098. 1098. 1098.] делает один общ список

# искомая матрица  flatten запихивает масив м строку(но массив одинарный) F порядок записи первые элементы потом вторые
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0], # [55.0] * 3363580 количество элементов записсанные в строку
                                ## [1, 1, 1,... 1, 1, 1, 1]
                                sites_flatten,# САМ МАСИВ
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]#от 0 10 20 30 40 до 3363590 с шагам 10 
                                # [:, 1:]# весь второй столбец и того
    #[1, 1, 1,... 1, 1, 1, 1] , 3363580 , [0, 10, 20..336590] , и второй столбец
#print(full_sites_sparse)

#print(full_sites_sparse)
#full_sites_sparse.toarray()
#x_train_shape = full_sites_sparse[:idx_split]
#x_test_shape = full_sites_sparse[idx_split]
#sparse = csr_matrix(matri)
#sparse.shape
#full_sites_sparse

In [39]:
 from sklearn.feature_extraction.text import CountVectorizer

In [73]:
cv = CountVectorizer()

In [90]:
t_train = cv.fit_transform(train_df[sites])
y_test = cv.transform(test_df[sites])
#print(t_train.shape, y_test.shape)
cv.fit_transform(train_df[full_sites])

ValueError: Must pass DataFrame with boolean values only

In [189]:
sites_flatten[0]*sites_flatten.shape[0]
sites_flatten.shape[0]   
datar = np.arange(1,11)
[1]*datar.shape[0]
a =range(0,datar.shape[0]+10 , 10)
[1]*datar.shape[0]
datar.shape[0]
type(a)
#vv = csr_matrix(([1] * datar.shape[0], # [55.0] * 3363580 количество элементов записсанные в строку
                                ## [1, 1, 1,... 1, 1, 1, 1]
#                                datar,
                                
#                ))#от 0 до 3363590 с шагам 10 

range

In [64]:

def get_auc_lr_valid(x, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
   
    train_len = int(ratio * x.shape[0]) #90 процентов выборки от исхода index[0] пока х его но это хитрая матрица
   # t_train = x[:train_len]
    x_train = x[:train_len,:]
    x_valid = x[train_len:,:]# : означает что мы берем все признаки
    y_train = y[:train_len]
    y_valid = y[train_len:]
    logit = LogisticRegression(C=C,n_jobs=1,random_state=seed,)
    logit.fit(x_train, y_train)
    valid_pred = logit.predict_proba(x_valid)[:,1]# отличае от predict то что выдает предсказан вероятности а не метки 0 и 1
            # [:,1] это второй столбец со значениями + то что это элис
#    valid_pred = logit.predict(x_valid)# отличае от predict то что выдает предсказан вероятности а не метки 0 и 1
    return roc_auc_score(y_valid, valid_pred) # провер на сколько точен прогноз в виде вероятности roc_auc

#print(x_train_shape)    
#print(x_train_shape)
#print(x_train_shape[:1])


In [197]:
logit = LogisticRegression(n_jobs=1, random_state=17)
logit.fit(x_train_shape,y_train)
    
#
#logit.predict_proba(x_train_shape[:15,:])[:,1]# так покажет первые 15 и где вераятность ближе к 1
rrr = logit.predict_proba(x_train_shape)[:,1]# так покажет первые 15 и где вераятность ближе к 1   # [:,1]вероятность того что это Alice
rrr =get_auc_lr_valid(x_train_shape, y_train)

In [198]:
new_feat_train = pd.DataFrame(index = train_df.index) # Берет фрэим train_df и заберает только индексы
new_feat_test = pd.DataFrame(index = train_df.index) # Берет фрэим train_df и заберает только индексы
new_feat_train.head(3)


""
session_id
21669
54843
77292


In [201]:
new_feat_train['year_month'] = train_df['time1'].apply(lambda ts:100*ts.year+ ts.month)# короч к году *100 + месяц 1 = 20130 +1 = 201301
new_feat_test['year_month'] = train_df['time1'].apply(lambda ts:100*ts.year+ ts.month)# короч к году *100 + месяц 1 = 20130 +1 = 201301


In [175]:
scaler = StandardScaler() # С каждого столбца вычтает среднее и делет на стандартное отклонение
scaler.fit(new_feat_train['year_month'].values.reshape(-1, 1))
new_feat_train['year_month_scaled']=scaler.transform(new_feat_train['year_month'].values.reshape(-1, 1))
new_feat_test['year_month_scaled']=scaler.transform(new_feat_train['year_month'].values.reshape(-1, 1))

In [176]:
x_train_sparse_new = csr_matrix(hstack([x_train_shape, new_feat_train['year_month_scaled'].values.reshape(-1, 1)]))
get_auc_lr_valid(x_train_sparse_new, y_train)

c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9196935128594357

In [177]:
x_train_shape.shape, new_feat_train.values.reshape(-1,1).shape

((253561, 48371), (507122, 1))

# Log Regresion on CounteVector

In [8]:
from sklearn.model_selection import cross_val_score
from  sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()  

with open('train_text.txt') as inp_train_file:
    tr_train = vectorizer.fit_transform(inp_train_file)
  
with open('test_text.txt') as inp_test_file:
    ts_test = vectorizer.transform(inp_test_file)    

print(tr_train.shape, ts_test.shape)

(253561, 41592) (82797, 41592)


In [303]:
def write_to_submiss_file(predict_labels, out_file,target='target',index_label='session_id'):
    predicted_df = pd.DataFrame(predict_labels, index = np.arange(1, predict_labels.shape[0]+1),columns=[target])
    predicted_df.to_csv(out_file, index_label= index_label)

In [9]:
yy_train = train_df['target'].astype('int')

In [10]:
logit1 = LogisticRegression(C=1,random_state=17)

In [11]:
sc_score =cross_val_score(logit1, tr_train, yy_train, cv=5, scoring='roc_auc')

c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-pa

In [12]:
sc_score.mean()

0.8851554065765168

In [251]:
logit1.fit(tr_train, yy_train)

c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [301]:
test_predict_lofReg = logit1.predict_proba(ts_test)[:,1]# [:,1]вероятность того что это Alice
test_predict_lofReg

array([2.42986455e-03, 5.10600105e-09, 1.88419120e-08, ...,
       8.84323536e-03, 4.74556494e-04, 2.26304918e-05])

In [304]:
#len(test_predict_lofReg)
write_to_submiss_file(test_predict_lofReg, 'logit_submit.txt')

# add features time

In [311]:
from scipy.sparse import hstack

In [405]:

def add_time_fetures(time1_sirees, x_sparse):
    hour = time1_sirees.apply(lambda ts: ts.hour) # час от 0 до 23
    morning =((hour>=6) & (hour <= 11)).astype('int')
    day =((hour>=12) & (hour <= 18)).astype('int')
    evening =((hour>=18) & (hour <= 0)).astype('int')
    night =((hour>=1) & (hour <= 4)).astype('int')
    X = hstack([x_sparse,
                morning.values.reshape(-1,1),
                day.values.reshape(-1,1),
                evening.values.reshape(-1,1),
                night.values.reshape(-1,1)])
    return X

In [406]:
x_tr_with_tim1 = add_time_fetures(train_df.time1.fillna(0),tr_train)

In [337]:
logitTime1 = LogisticRegression(C=1.9,random_state=17)

In [346]:
sc_score_time =cross_val_score(logitTime1, x_tr_with_tim1, yy_train, cv=5, scoring='roc_auc')

c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\helg\appdata\local\programs\python\python37\lib\site-pa

In [347]:
sc_score_time.mean()

0.9306763159423495

In [348]:
logitTime1.fit(x_tr_with_tim1,yy_train)

c:\users\helg\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.9, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [349]:
test_time1_predict_lofReg = logitTime1.predict_proba(x_ts_with_tim1)[:,1]
test_time1_predict_lofReg

array([3.32117418e-05, 1.22199899e-08, 6.53345442e-09, ...,
       2.16053917e-04, 5.70165335e-06, 2.04523332e-07])

In [350]:
write_to_submiss_file(test_time1_predict_lofReg, 'logit_ti_submit.txt')

## all time

In [417]:
%%time
x_tr_with_tim12 = add_time_fetures(train_df.time2.fillna(0),tr_train)
x_ts_with_tim12 = add_time_fetures(test_df.time2.fillna(0), ts_test)
print(x_tr_with_tim1.shape, x_ts_with_tim1.shape)

AttributeError: 'int' object has no attribute 'hour'

In [ ]:
u = train_df.time1.apply(lambda ts: ts.hour)
for i in u:
    if  ((u>=6) & (u <= 11)) :
    print(u)

In [279]:
 np.arange(6).reshape(-1,1) # Делает столбец векторный

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])